# Load User Models data

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
import warnings
warnings.filterwarnings('ignore')

In [40]:
import os
from ast import literal_eval
import pandas as pd
import numpy as np

In [41]:
os.chdir("/content/drive/MyDrive/MovieRecommendationSystem")

In [42]:
with open("dataset/ModelStorage/idToTitle.json", "r") as file:
    idToTitle = file.read()

In [43]:
ratings = pd.read_csv("dataset/ratings_small.csv")

In [44]:
idToTitle = literal_eval(idToTitle)


In [45]:
idToTitle = { int(k):v for k, v in idToTitle.items()}

In [46]:
idToTitle[949]

'Heat'

In [47]:
dataset = pd.read_csv("dataset/FeatureExtracted/dataset.csv")

In [48]:
dataset.drop('id', axis = 1, inplace = True)

In [49]:
with open("dataset/ModelStorage/indexToId.txt", "r") as file:
    indexToId = file.read()

In [50]:
indexToId = literal_eval(indexToId)

In [51]:
with open("dataset/ModelStorage/idToIndex.txt", "r") as file:
    idToIndex = file.read()

In [52]:
idToIndex = literal_eval(idToIndex)

In [53]:
U = np.fromfile("dataset/ModelStorage/U.txt").reshape((671,38))
b = np.fromfile("dataset/ModelStorage/b.txt").reshape((671,1))

# Prediction
Fist we need to calculate the matrix of ratings (Same step as in Machine Learning Phase of Classification Model)

In [54]:
from sklearn import preprocessing
dataset = preprocessing.scale(dataset)

In [55]:
dataset = np.concatenate((dataset, np.ones((2830,1))), axis = 1)

In [56]:
probs_matrix = np.matmul(np.concatenate((U,b), axis = 1), dataset.T)

In [57]:
def sigmoid(x):
    return 1 /(1 + 1 / np.exp(x))

In [58]:
probs_matrix = sigmoid(probs_matrix)

# Recommendation
Let's specify user id (From 0 to 670)

In [59]:
user = 10

In [60]:
probs_matrix[user - 1]

array([0.65606782, 0.64573066, 0.63895139, ..., 0.64690548, 0.63064072,
       0.60526062])

In [61]:
decision_user_df = pd.DataFrame()

In [62]:
decision_user_df['movie_Id'] = list(range(probs_matrix[user - 1].size))
decision_user_df['movie_Id'] = decision_user_df['movie_Id'].apply(lambda x: indexToId[x])
decision_user_df['shouldRecommend'] = probs_matrix[user - 1]

Transform the probabilty: 1 if p>=0.5 else 0

1 means that user may rate 4.0 for the movie

0 means lower than 4.0

In [63]:
decision_user_df['shouldRecommend'] = decision_user_df['shouldRecommend'].apply(lambda x: 1 if x>=0.5 else 0)

In [64]:
decision_user_df

,movie_Id,shouldRecommend
0,949,1
1,710,1
2,1408,1
3,524,1
4,4584,1
...,...,...
2825,80831,1
2826,3104,1
2827,64197,1
2828,98604,1


In [65]:
decision_user_df['shouldRecommend'].value_counts()

1    2830
Name: shouldRecommend, dtype: int64

In the predicted ***shouldRecommend***(pred rating >=4) for user, drop the movies that they have seen

In [66]:
movies_have_seen = ratings[ratings['userId'] == user]['movieId'].values

In [67]:
decision_user_df[decision_user_df['movie_Id'].isin(movies_have_seen)].index

Int64Index([ 286,  382,  411,  524,  609,  636,  685,  747,  783, 1078, 1216,
            1272, 1414, 1436, 1441, 1487, 1610, 1658, 1660, 1821, 2690],
           dtype='int64')

In [68]:
decision_user_df.drop(decision_user_df[decision_user_df['movie_Id'].isin(movies_have_seen)].index, inplace = True)

In [69]:
indexToRecommend = decision_user_df['shouldRecommend'].sort_values(ascending = False)[0:20].index

Movie user have seen.

In [70]:
for x in movies_have_seen:
    if x in indexToId.keys():
        if indexToId[x] in idToTitle.keys():
            print(idToTitle[indexToId[x]])

Leon: The Professional
The Maltese Falcon
Con Air
Scream 2
Titanic
Backdraft
The Gift
Shattered
Mystic River
In My Skin
Aguirre: The Wrath of God
Van Helsing
Swimming to Cambodia
The Rolling Stones: Gimme Shelter
Mercy
The Wages of Fear
Zardoz
Spring, Summer, Fall, Winter... and Spring
The Leopard
Fires on the Plain
The Tunnel
Charlie and the Chocolate Factory
Cube Zero
Underworld: Evolution
Land of Plenty
The Last King of Scotland
Blood Diamond
In the Name of the King: A Dungeon Siege Tale
The Happening
The Haunted Castle
Samaritan Zatôichi
Hotel Very Welcome
The Message
Carry On Doctor
Devil in the Flesh
Frankenstein Created Woman
Travelling with Pets


Movie to recommend user

In [71]:
for x in indexToRecommend:
    print(idToTitle[indexToId[x]])

Heat
The Bank Job
The Return of Doctor X
That Man from Rio
Clean, Shaven
À nos amours
Canvas
Run, Fatboy, Run
The Marksman
10,000 BC
Hallam Foe
The Mostly Unfabulous Social Life of Ethan Green
City of Men
The Naked Island
Downhill Racer
Mala Noche
The Dark Knight
Phar Lap
The Promised Land
Khadak
